# March Madness Monte Carlo Simulation

Use information to try to predict the 2023 March Madness Men's and Women's Basketball Championships.

## Configure Notebook and Download Competition Items

In [1]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [2]:
from pathlib import Path
import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/march-machine-learning-mania-2023')
    !pip install -Uqq fastai
else:
    import zipfile,kaggle
    path = Path('../march-machine-learning-mania-2023')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

## Load and Preview Provided Data

In [3]:
import pandas as pd

# general datasets
cities = pd.read_csv(path/'Cities.csv')
conferences = pd.read_csv(path/'Conferences.csv')
samp_submission = pd.read_csv(path/'SampleSubmission2023.csv')

# men's datasets
m_conf_tourney_games = pd.read_csv(path/'MConferenceTourneyGames.csv')
m_game_cities = pd.read_csv(path/'MGameCities.csv')
m_massey_ordinals = pd.read_csv(path/'MMasseyOrdinals_thru_Season2023_Day128.csv')
m_tourney_compact_results = pd.read_csv(path/'MNCAATourneyCompactResults.csv')
m_tourney_detailed_results = pd.read_csv(path/'MNCAATourneyDetailedResults.csv')
m_tourney_seed_round_slots = pd.read_csv(path/'MNCAATourneySeedRoundSlots.csv')
m_tourney_seeds = pd.read_csv(path/'MNCAATourneySeeds.csv')
m_tourney_slots = pd.read_csv(path/'MNCAATourneySlots.csv')
m_reg_season_compact_results = pd.read_csv(path/'MRegularSeasonCompactResults.csv')
m_reg_season_detailed_results = pd.read_csv(path/'MSeasons.csv')
m_secondary_tourney_compact_results = pd.read_csv(path/'MSecondaryTourneyCompactResults.csv')
m_secondary_tourney_teams = pd.read_csv(path/'MSecondaryTourneyTeams.csv')
m_team_coaches = pd.read_csv(path/'MTeamCoaches.csv')
m_team_conferences = pd.read_csv(path/'MTeamConferences.csv')
m_teams = pd.read_csv(path/'MTeams.csv')
m_team_spellings = pd.read_csv(path/'MTeamSpellings.csv', encoding='latin-1')

# women's datasets
w_game_cities = pd.read_csv(path/'WGameCities.csv')
w_tourney_compact_results = pd.read_csv(path/'WNCAATourneyCompactResults.csv')
w_tourney_detailed_results = pd.read_csv(path/'WNCAATourneyDetailedResults.csv')
w_tourney_seeds = pd.read_csv(path/'WNCAATourneySeeds.csv')
w_tourney_slots = pd.read_csv(path/'WNCAATourneySlots.csv')
w_reg_season_compact_results = pd.read_csv(path/'WRegularSeasonCompactResults.csv')
w_reg_season_detailed_results = pd.read_csv(path/'WRegularSeasonDetailedResults.csv')
w_seasons = pd.read_csv(path/'WSeasons.csv')
w_team_conferences = pd.read_csv(path/'WTeamConferences.csv')
w_teams = pd.read_csv(path/'WTeams.csv')
w_team_spellings = pd.read_csv(path/'WTeamSpellings.csv', encoding='latin-1')

In [4]:
m_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023


## Initial Data Preparation
### Tournament Datasets

#### Concatenate Men's and Women's Tournament Results

In [5]:
# add column to identify whether it is men's or women's 
m_tourney_compact_results['gender'] = 'men'
w_tourney_compact_results['gender'] = 'women'
m_tourney_detailed_results['gender'] = 'men'
w_tourney_detailed_results['gender'] = 'women'

# concat datasets together
tourney_compact = pd.concat([m_tourney_compact_results, w_tourney_compact_results], axis=0)
tourney_detailed = pd.concat([m_tourney_detailed_results, w_tourney_detailed_results], axis=0)

#### Add Conference Detail

In [6]:
# concat mens and womens datasets
team_conferences = pd.concat([m_team_conferences, w_team_conferences], axis=0)

In [7]:
# create copy of compact results dataset for now
df = tourney_compact.copy()

In [8]:
# add winning team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'WTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

# add losing team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'LTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

#### Concat City Detail

In [9]:
# concat mens and womens datasets
game_cities = pd.concat([m_game_cities, w_game_cities], axis=0)

In [10]:
# merge CRType and CityID into df
df = df.merge(game_cities, how='left', on=['Season', 'DayNum', 'WTeamID', 'LTeamID'])

#### Concat Tourney Seed Detail

In [11]:
# concat men's and women's datasets
tourney_seeds = pd.concat([m_tourney_seeds, w_tourney_seeds], axis=0)

In [12]:
# concat winning seed data
df = df.merge(tourney_seeds, how='left', left_on=['WTeamID', 'Season'], right_on=['TeamID', 'Season']).rename(columns={'Seed':'WTeamSeed'})
df = df.drop('TeamID', axis=1)

In [13]:
# concat losing seed data
df = df.merge(tourney_seeds, how='left', left_on=['LTeamID', 'Season'], right_on=['TeamID', 'Season']).rename(columns={'Seed':'LTeamSeed'})
df = df.drop('TeamID', axis=1)

#### Save Copy of Merged Tourney Data

In [14]:
tourney_df = df.copy()

### Regular Season Data

#### Concat Men's and Women's Regular Season Data

In [15]:
# add gender column to compact and detailed dataframes
m_reg_season_compact_results['gender'] = 'men'
m_reg_season_detailed_results['gender'] = 'men'
w_reg_season_compact_results['gender'] = 'women'
w_reg_season_detailed_results['gender'] = 'women'

In [16]:
# concat men's and women's regular season data
reg_season_compact = pd.concat([m_reg_season_compact_results, w_reg_season_compact_results], axis=0)
reg_season_detailed = pd.concat([m_reg_season_detailed_results, w_reg_season_detailed_results], axis=0)

#### Concat Conference Detail

In [17]:
team_conferences.head()

,Season,TeamID,ConfAbbrev
0,1985,1102,wac
1,1985,1103,ovc
2,1985,1104,sec
3,1985,1106,swac
4,1985,1108,swac


In [18]:
# save dataframe for compact reg season
df = reg_season_compact.copy()

In [19]:
# add winning team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'WTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

# add losing team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'LTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

#### Concat City Detail

In [20]:
# merge CRType and CityID into df
df = df.merge(game_cities, how='left', on=['Season', 'DayNum', 'WTeamID', 'LTeamID'])

#### Save Regular Season Data

In [21]:
reg_season_df = df.copy()

## Feature Engineering

In [22]:
reg_season_df.head(3)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID
0,1985,20,1228,81,1328,64,N,0,men,big_ten,big_eight,NaN,NaN
1,1985,25,1106,77,1354,70,H,0,men,swac,meac,NaN,NaN
2,1985,25,1112,63,1223,56,H,0,men,pac_ten,a_sun,NaN,NaN


In [23]:
tourney_df.head(3)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed
0,1985,136,1116,63,1234,54,N,0,men,swc,big_ten,NaN,NaN,X09,X08
1,1985,136,1120,59,1345,58,N,0,men,sec,big_ten,NaN,NaN,Z11,Z06
2,1985,136,1207,68,1250,43,N,0,men,big_east,ecc,NaN,NaN,W01,W16


#### Wins by Team Each Season

In [24]:
season_summarized = reg_season_df.groupby(['Season', 'WTeamID']).count()['WScore'].reset_index()

In [29]:
# rename columns 
season_summarized.columns = ['Season', 'TeamID', 'reg_season_wins']

#### Losses by Team Each Season

In [30]:
losses = reg_season_df.groupby(['Season', 'LTeamID']).count()['LScore'].reset_index()
losses.columns=['Season', 'TeamID', 'reg_season_losses']

In [33]:
# merge into season_summarized
season_summarized = season_summarized.merge(losses, how='left', on=['Season', 'TeamID'])

In [34]:
season_summarized.head()

,Season,TeamID,reg_season_wins,reg_season_losses
0,1985,1102,5,19.0
1,1985,1103,9,14.0
2,1985,1104,21,9.0
3,1985,1106,10,14.0
4,1985,1108,19,6.0


In [28]:
circuit breaker

SyntaxError: invalid syntax (3229717645.py, line 1)

## Preprocess Datasets

In [ ]:
# preview format of sample submissions
samp_submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


We know the Pred column is the probability that the first team beats the second team. 

In [ ]:
# merge regular season and tournament data
df_concat = pd.concat([reg_season_df, tourney_df], axis=0)

In [ ]:
df_concat.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ,gender,DayNum,WTeamID,WScore,...,LTO,LStl,LBlk,LPF,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed
0,1985,10/29/1984,East,West,Midwest,Southeast,men,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986,10/28/1985,East,Midwest,Southeast,West,men,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1987,10/27/1986,East,Southeast,Midwest,West,men,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1988,11/2/1987,East,Midwest,Southeast,West,men,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1989,10/31/1988,East,West,Midwest,Southeast,men,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# create a copy of df_concat to work with
df_pre = df_concat.copy()

In [ ]:
# add ID column matching sample submission
df_pre['ID_winner_first'] = df_pre.apply(lambda x: f'{x.Season}_{x.WTeamID}_{x.LTeamID}', axis=1)
df_pre['ID_loser_first'] = df_pre.apply(lambda x: f'{x.Season}_{x.LTeamID}_{x.WTeamID}', axis=1)

In [ ]:
df_pre.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed,ID_winner_first,ID_loser_first
0,1985,20,1228,81,1328,64,N,0,men,big_ten,big_eight,NaN,NaN,NaN,NaN,1985_1228_1328,1985_1328_1228
1,1985,25,1106,77,1354,70,H,0,men,swac,meac,NaN,NaN,NaN,NaN,1985_1106_1354,1985_1354_1106
2,1985,25,1112,63,1223,56,H,0,men,pac_ten,a_sun,NaN,NaN,NaN,NaN,1985_1112_1223,1985_1223_1112
3,1985,25,1165,70,1432,54,H,0,men,ivy,ind,NaN,NaN,NaN,NaN,1985_1165_1432,1985_1432_1165
4,1985,25,1192,86,1447,74,H,0,men,ecacm,ecacm,NaN,NaN,NaN,NaN,1985_1192_1447,1985_1447_1192


In an effort to prevent overfitting to the format provided, we will divide the DataFrame in 2. One half will present winners first, with `Pred` set equal to 1. The remaining half will present losers first, with `Pred` set equal to 0. 

In [ ]:
# split datasets in two
df_pre_a = df_pre.sample(frac=0.5)
df_pre_b = df_pre.drop(df_pre_a.index)

In [ ]:
# reset indices
df_pre_a = df_pre_a.reset_index(drop=True)
df_pre_b = df_pre_b.reset_index(drop=True)

 - `df_pre_a` -- winner presented first within ID
 - `df_pre_b` -- loser presented first within ID

In [ ]:
# create proper ID and Pred Columns
df_pre_a['ID'] = df_pre_a['ID_winner_first']
df_pre_a['Pred'] = 1
df_pre_b['ID'] = df_pre_b['ID_loser_first']
df_pre_b['Pred'] = 0

In [ ]:
# drop helper columns
df_pre_a = df_pre_a.drop(['ID_winner_first', 'ID_loser_first'], axis=1)
df_pre_b = df_pre_b.drop(['ID_winner_first', 'ID_loser_first'], axis=1)

In [ ]:
# merge datasets back together
df_preprocessed = pd.concat([df_pre_a, df_pre_b], axis=0)

In [ ]:
df_preprocessed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed,ID,Pred
0,2008,116,3171,46,3165,44,H,0,women,ivy,ivy,NaN,NaN,NaN,NaN,2008_3171_3165,1
1,2019,4,3234,90,3331,77,H,0,women,big_ten,summit,Regular,4162.0,NaN,NaN,2019_3234_3331,1
2,2010,112,1370,90,1363,88,A,0,men,ind,wac,Regular,4305.0,NaN,NaN,2010_1370_1363,1
3,2008,11,1160,58,1366,43,N,0,men,big_twelve,ind,NaN,NaN,NaN,NaN,2008_1160_1366,1
4,2020,96,3153,73,3374,58,A,0,women,aac,aac,Regular,4088.0,NaN,NaN,2020_3153_3374,1


Now that we have a combined dataset with the proper columns needed for submission, we can continue with EDA and begin building out our classification model. 

In [ ]:
# save initial preprocessed dataset
data_files = path/'..'/'data'
df_preprocessed.to_csv(data_files/'initial_preprocessed.csv', index=False)